In [1]:
#導入pandas、numpy、os套件
import pandas as pd
import numpy as np
import os

#訓練料資料前處理
G='大數據競賽初賽資料(230測試數據)\\'         #訓練集路徑
G_path=os.listdir(G)                     #用os.listdir 把"大數據競賽初賽資料(230測試數據)"下的檔案放入G_path
txt_dat_list=list()                
for i in G_path:
    cnt=0
    txt_path=os.listdir(G+i)            #用os.listdir 把(G11、G15、G17、G19、G32、G34、G48、G49)資料夾下的txt放入list
    for j in txt_path:
        #print(G+"\\"+i+"\\"+j)
        txt_dat=G+"\\"+i+"\\"+j             #設定txt的完整路徑
        txt_data=pd.read_table(txt_dat)        #用pd.read_table 讀進txt
        txt_data = txt_data[txt_data.columns[:-1]]
        txt_data=txt_data.drop(txt_data.index[[0]])               #移除txt中的 Deg.F
        df_obj = txt_data.select_dtypes(['object'])
        txt_data[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
        #print(j+'\t'+str(txt_data.shape))                                 
        for k in txt_data:
            txt_dat_list.append([txt_data[k]])                #把一個sensor當作一筆訓練資料放進txt_dat_list裡
            cnt+=1
    print(i+" txt quantity : "+str(cnt))
print("All txt : "+str(np.array(txt_dat_list).shape))           # All txt : (1745, 1)

C:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  app.launch_new_instance()


G11 txt quantity : 145
G15 txt quantity : 207
G17 txt quantity : 119
G19 txt quantity : 238
G32 txt quantity : 256
G34 txt quantity : 264
G48 txt quantity : 244
G49 txt quantity : 272
All txt : (1745, 1)


In [2]:
#導入sequences套件
from keras.preprocessing import sequence

cnt=0
for i in txt_dat_list:
    txt_dat_list[cnt]=sequence.pad_sequences(i,dtype='float64',maxlen=449)          #統一sensor溫度陣列的長度
    cnt+=1
print(str(np.array(txt_dat_list).shape))                                   #(1745, 1, 449)
txt_dat_list = np.reshape(txt_dat_list,(-1,449))                           #三維轉二維
print(str(np.array(txt_dat_list).shape))                                   #(1745, 449)

Using TensorFlow backend.


(1745, 1, 449)
(1745, 449)


In [3]:
#導入shuffle套件
from sklearn.utils import shuffle

#設定labels值
num_of_samples=txt_dat_list.shape[0]
labels=np.zeros((num_of_samples,),dtype='int64')
labels[0:145]=11
labels[145:352]=15
labels[352:471]=17
labels[471:709]=19
labels[709:965]=32
labels[965:1229]=34
labels[1229:1473]=48
labels[1473:1745]=49
#print(labels[:5])

#將溫度曲線特徵向量與標籤對應
x_train,y_train=shuffle(txt_dat_list,labels,random_state=2)

In [4]:
#導入train_test_split,cross_val_score套件及LogisticRegression模型
#使用LogisticRegression模型訓練資料並且使用交叉驗證測試準確率
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,cross_val_score

x_train,x_test,y_train,y_test=train_test_split(x_train,y_train,test_size=0.2,random_state=500)      
classifier=LogisticRegression(solver='newton-cg')              #設定LogisticRegression參數

score=cross_val_score(classifier,x_train,y_train,cv=5)         #交叉驗證
print("Cross val score :",score * 100)
print("Average cross val score : ",score.mean() * 100)
classifier.fit(x_train,y_train)                              #進行模型訓練

#test_target= classifier.predict(x_test)
#print(test_target)
#print(y_test)

C:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
C:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
C:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
C:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-

Cross val score : [ 98.58156028  98.92857143  99.64285714 100.          99.27536232]
Average cross val score :  99.28567023479144


C:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
C:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
C:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\scipy\optimize\linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\scipy\optimize\linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. In

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)

In [5]:
!pip install xlrd

In [6]:
!pip install openpyxl

In [7]:
import re
import xlrd
import openpyxl

G_test='thubigdata2019exam-722\\'                    #設定測試資料的路徑
G_test_path=os.listdir(G_test)                    #用os.listdir 把"thubigdata2019exam-722"下的檔案放入G_test_path
answer_xlsx = pd.read_excel("108106_TestResult.xlsx")      #讀進108106_TestResult.xlsx

for i in G_test_path:
    test_txt_list=list()
    if i!="__MACOSX":                        #判斷是否為txt檔
        test_txt_path=G_test+i            #設定txt的完整路徑
        test_txt_data=pd.read_table(test_txt_path,delim_whitespace=True)           #用pd.read_table 讀進txt
        test_txt_data=test_txt_data.drop(test_txt_data.index[[0]])                #移除txt中的 Deg.F
        #print(i+'\t'+str(test_txt_data.shape))  
        for j in test_txt_data:
            test_txt_list.append([test_txt_data[j]])                          #將每個sensor加入test_txt_list
        #print(str(i)+"\t"+str(np.array(test_txt_list).shape))                        
            cnt=0
            for k in test_txt_list:
                test_txt_list[cnt]=sequence.pad_sequences(k,dtype='float64',maxlen=449)         #統一sensor溫度陣列的長度
                cnt+=1
        #print(str(i)+"\t"+str(np.array(test_txt_list).shape))
        test_txt_list = np.reshape(test_txt_list,(-1,449))                      #三維轉二維
        #print(str(i)+"\t"+str(np.array(test_txt_list).shape))
        prediction=classifier.predict(test_txt_list)                      #預測測試集
        print(str(i)+"\t"+str(np.array(test_txt_list).shape)," prediction_array : ",prediction)#印出每個txt中每個sensor的預測結果
        count = np.bincount(prediction)            #計算prediction中每個數值的出現次數
        answer = np.argmax(count)                #將出現次數最多次的數值指定給變數answer
        answer="G"+str(answer)
        txt_name=re.split(r'.txt',i)[0]
        print(txt_name+" "+"answer : ",answer)      #印出最後答案
        
        #將答案寫進excel裡
        for n in answer_xlsx['測驗數據資料代號']:
            if txt_name==str(n):
                answer_xlsx['分類結果'][int(txt_name)-1]=answer
        print('..............................')
answer_xlsx.to_excel("108106_TestResult.xlsx",index=False)

C:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  del sys.path[0]
C:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  del sys.path[0]
C:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launc

1.txt	(5, 449)  prediction_array :  [11 11 11 11 11]
1 answer :  G11
..............................
10.txt	(8, 449)  prediction_array :  [17 17 17 17 17 17 17 17]
10 answer :  G17
..............................
11.txt	(8, 449)  prediction_array :  [19 19 19 19 19 19 19 19]
11 answer :  G19
..............................
12.txt	(8, 449)  prediction_array :  [19 19 19 19 19 19 19 19]
12 answer :  G19


C:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  del sys.path[0]
C:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  del sys.path[0]
C:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launc

..............................
13.txt	(8, 449)  prediction_array :  [32 32 32 32 32 32 32 32]
13 answer :  G32
..............................
14.txt	(8, 449)  prediction_array :  [32 32 32 32 32 32 32 32]
14 answer :  G32
..............................
15.txt	(8, 449)  prediction_array :  [32 32 32 32 32 32 32 32]
15 answer :  G32
..............................
16.txt	(8, 449)  prediction_array :  [32 32 32 32 32 32 32 32]
16 answer :  G32


C:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  del sys.path[0]
C:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  del sys.path[0]
C:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launc

...

C:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  del sys.path[0]
C:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  del sys.path[0]
C:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launc

..............................
36.txt	(8, 449)  prediction_array :  [49 49 49 49 49 49 49 49]
36 answer :  G49
..............................
4.txt	(6, 449)  prediction_array :  [15 15 15 15 15 15]
4 answer :  G15
..............................
5.txt	(6, 449)  prediction_array :  [15 15 15 15 15 15]
5 answer :  G15
..............................
6.txt	(8, 449)  prediction_array :  [19 15 15 15 15 15 15 15]
6 answer :  G15


C:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  del sys.path[0]
C:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  del sys.path[0]
C:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launc

..............................
7.txt	(6, 449)  prediction_array :  [15 15 15 15 15 15]
7 answer :  G15
..............................
8.txt	(8, 449)  prediction_array :  [15 15 15 15 15 15 15 15]
8 answer :  G15
..............................
9.txt	(8, 449)  prediction_array :  [17 17 17 17 17 17 17 17]
9 answer :  G17
..............................


C:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [8]:
#答案總覽
answer_xlsx

,測驗數據資料代號,分類結果
0,1,G11
1,2,G11
2,3,G15
3,4,G15
4,5,G15
5,6,G15
6,7,G15
7,8,G15
8,9,G17
9,10,G17
